The following is a simpler version of the geopandas assignment on US shootings near schools. Here the analysis is reduced to things that may be useful for an app. The app is not yet built, but the data and analysis is prepared for it.
In addition Folium is used to map out things. Folium integrates well with Streamlit and therefore a good thing to practice. Some fancier things are added in folium to present you with some options (e.g. custom tiles, marcer clusters, popups, etc.).

In [1]:
import geopandas as gpd
import pandas as pd
from folium.plugins import MarkerCluster
import folium

In [2]:

police = pd.read_csv('https://raw.githubusercontent.com/aaubs/ds-master/main/data/geopandas_data/SPD_Officer_Involved_Shooting__OIS__Data.csv')
gdf_ps = gpd.read_file('https://raw.githubusercontent.com/aaubs/ds-master/main/data/Public_Schools.geojson')
gdf = gpd.read_file('https://raw.githubusercontent.com/aaubs/ds-master/main/data/geopandas_data/Neighborhood_Map_Atlas_Districts.geojson')

In [6]:
# Convert the police dataframe to a geodataframe
gdf_police = gpd.GeoDataFrame(police, geometry=gpd.points_from_xy(police['Longitude'], police['Latitude']))
gdf_police.crs = "EPSG:4326"

In [7]:
# Update CRS for all geo dataframes
gdf_police = gdf_police.to_crs("EPSG:4326")
gdf = gdf.to_crs("EPSG:4326")
gdf_ps = gdf_ps.to_crs("EPSG:4326")

# Spatial joins for neighborhoods and schools with police data
joined_police_gdf = gpd.sjoin(gdf_police, gdf, how="left", predicate="within")
joined_police_ps = gdf_police.sjoin_nearest(gdf_ps, how='left')

/opt/miniconda3/envs/streamlit/lib/python3.9/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


## Spatial Distribution of Data

In [8]:
# Initialize a map centered on Seattle
m = folium.Map(location=[47.6062, -122.3321], zoom_start=11, tiles="CartoDB positron")

# Use the MarkerCluster plugin to cluster the points (optional, when there are many points)
marker_cluster = MarkerCluster().add_to(m)

# Iterate through the joined police data and add a marker for each incident
for _, row in gdf_police.iterrows():
    popup_content = f"""
    Incident Number: {row['Incident Number']}<br>
    Date/Time: {row['Date / Time']}<br>
    Address: {row['Blurred Address']}<br>
    Officer Rank: {row['Rank']}<br>
    Officer Gender: {row['Officer Gender']}<br>
    Subject Gender: {row['Subject Gender']}<br>
    Subject Race: {row['Subject Race']}<br>
    """
    popup = folium.Popup(popup_content, max_width=300)
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=15,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.4,
        popup=popup
    ).add_to(marker_cluster)

# to produce a HTML file
# m.save("Spatial_Distribution.html")

#to simply display the map
m

## Common Neighborhoods for Police-Involved Shootings

In [9]:
joined_police_gdf.L_HOOD.value_counts()

L_HOOD
Downtown               35
Northeast              28
Rainier Valley         24
Central Area           14
Cascade                13
Northgate              11
Capitol Hill            9
Northwest               9
Queen Anne              7
West Seattle            6
Greater Duwamish        5
Beacon Hill             5
University District     3
North Central           2
Ballard                 1
Magnolia                1
Name: count, dtype: int64

In [ ]:
# Subject Race

gdf_police['Subject Race'].value_counts()

## Neighborhoods Without Police-Involved Shootings

In [10]:
no_shooting_neighborhoods = gdf[~gdf.L_HOOD.isin(joined_police_gdf.L_HOOD.unique().tolist())]

m = folium.Map(location=[47.6062, -122.3321], zoom_start=11, tiles="CartoDB positron")
no_shooting_neighborhoods = no_shooting_neighborhoods.to_crs(epsg=4326)
for _, r in no_shooting_neighborhoods.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(r["L_HOOD"]).add_to(geo_j)
    geo_j.add_to(m)

m

## Schools Near Shooting Locations

In [12]:

joined_police_ps = gdf_police.sjoin_nearest(gdf_ps, how='left')
school_counts = joined_police_ps.NAME.value_counts()
school_counts

/opt/miniconda3/envs/streamlit/lib/python3.9/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


NAME
The Center School                 24
Bailey Gatzert                    19
Aki Kurose                        15
Bryant                            13
Lowell                            11
Sand Point                         9
TOPS K-8                           8
Broadview-Thomson K-8              8
Thurgood Marshall                  6
Robert Eagle Staff                 6
Rainier View                       6
Beacon Hill Int'l                  5
Queen Anne Gym                     4
Sacajawea                          4
Emerson                            4
Alki                               3
Leschi                             3
Roosevelt                          3
Greenwood                          3
BRIDGES                            3
McClure                            2
Lafayette                          2
Hawthorne                          2
Early Learning Center              2
Kimball - Interim                  2
Interagency at Columbia School     2
Washington                       

## Type of Schools Near Shootings

In [14]:
type_counts = joined_police_ps.TYPE.value_counts()
type_counts

TYPE
Elementary            108
Middle School          26
Option High School     24
Option Elementary      11
Service School          9
High School             6
NonStandard             4
Name: count, dtype: int64

## Schools Without Police-Involved Shootings

In [15]:
schools_without_shootings = gdf_ps[~gdf_ps.NAME.isin(joined_police_ps.NAME.unique().tolist())]
schools_without_shootings = schools_without_shootings.to_crs(epsg=4326)
m = folium.Map(location=[47.6062, -122.3321], zoom_start=11, tiles="CartoDB positron")

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Loop through each school and add it as a circle on the map within the marker cluster
for _, row in schools_without_shootings.iterrows():
    folium.Circle(
        location=[row.geometry.y, row.geometry.x],
        radius=20,  # you can adjust the radius
        color='violet',
        fill=True,
        fill_color='violet',
        fill_opacity=0.6,
        popup=row['SCHOOL']
    ).add_to(marker_cluster)

m

## School with Most Shootings

In [16]:
joined_police_ps_max = gdf_police.sjoin_nearest(gdf_ps, how='left', max_distance=0.09, distance_col="distances")
max_shooting_school = joined_police_ps_max.groupby(['NAME'])[['distances', 'NAME']].agg({'distances': 'mean', 'NAME': 'count'})
max_shooting_school

/opt/miniconda3/envs/streamlit/lib/python3.9/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,distances,NAME
NAME,,
Aki Kurose,0.002771,15
Alki,0.003770,3
BRIDGES,0.011776,3
Bailey Gatzert,0.015025,19
Beacon Hill Int'l,0.009482,5
Broadview-Thomson K-8,0.010852,8
Bryant,0.006201,13
Cascadia,0.005362,1
Catharine Blaine K-8,0.004638,1
